In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com//notebooks/dlsw-notebooks/remtting-started-session-based-03-serving-session-based-model-torch-backend/nvidia_logo.png" style="width: 90px; float: right;">

# Serving a Session-based Recommendation model with Torch Backend

This notebook is created using the latest stable [merlin-pytorch](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-pytorch/tags) container.

At this point, when you reach out to this notebook, we expect that you have already executed the `01-ETL-with-NVTabular.ipynb` and `02-session-based-XLNet-with-PyT.ipynb` notebooks, and saved the NVT workflow and the trained session-based model.

In this notebook, you are going to learn how you can serve a trained Transformer-based PyTorch model on NVIDIA [Triton Inference Server](https://github.com/triton-inference-server/server)  (TIS) with Torch backend using [Merlin systems](https://github.com/NVIDIA-Merlin/systems) library. One common way to do inference with a trained model is to use TorchScript, an intermediate representation of a PyTorch model that can be run in Python as well as in a high performance environment like C++. [TorchScript](https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html) is actually the recommended model format for scaled inference and deployment. TIS [PyTorch (LibTorch) backend](https://github.com/triton-inference-server/pytorch_backend) is designed to run TorchScript models using the PyTorch C++ API.

[Triton Inference Server](https://github.com/triton-inference-server/server) (TIS) simplifies the deployment of AI models at scale in production. TIS provides a cloud and edge inferencing solution optimized for both CPUs and GPUs. It supports a number of different machine learning frameworks such as TensorFlow and PyTorch.

### Import required libraries

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf
import glob
import torch 

from transformers4rec import torch as tr
from merlin.io import Dataset

from merlin.core.dispatch import make_df  # noqa
from merlin.systems.dag import Ensemble  # noqa
from merlin.systems.dag.ops.pytorch import PredictPyTorch  # noqa

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (NDCGAt). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state

We define the paths

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data")
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/sessions_by_day")
model_path= os.environ.get("model_path", f"{INPUT_DATA_DIR}/saved_model")

### Set the schema object

We create the schema object by reading the `schema.pbtxt` file generated by NVTabular pipeline in the previous, `01-ETL-with-NVTabular`, notebook.

In [4]:
from merlin_standard_lib import Schema
SCHEMA_PATH = os.environ.get("INPUT_SCHEMA_PATH", "/workspace/data/processed_nvt/schema.pbtxt")
schema = Schema().from_proto_text(SCHEMA_PATH)

We need to load the saved model to be able to serve it on TIS.

In [5]:
import cloudpickle
loaded_model = cloudpickle.load(
                open(os.path.join(model_path, "t4rec_model_class.pkl"), "rb")
            )

Switch the model to eval mode. We call `model.eval()` before tracing to set dropout and batch normalization layers to evaluation mode before running inference. Failing to do this might yield inconsistent inference results.

In [6]:
model = loaded_model.cuda()
model.eval()

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (continuous_module): SequentialBlock(
              (0): ContinuousFeatures(
                (filter_features): FilterFeatures()
                (_aggregation): ConcatFeatures()
              )
              (1): SequentialBlock(
                (0): DenseBlock(
                  (0): Linear(in_features=2, out_features=64, bias=True)
                  (1): ReLU(inplace=True)
                )
              )
              (2): AsTabular()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (item_id-list): Embedding(496, 64, padding_idx=0)
                (category-list): Embedding(179, 64, padding_idx=0)
              )
            )
          )
          (_aggregation): ConcatFeatures()
        

### Trace the model

We serve the model with the PyTorch backend that is used to execute TorchScript models. All models created in PyTorch using the python API must be traced/scripted to produce a TorchScript model. For tracing the model, we use [torch.jit.trace](https://pytorch.org/docs/stable/generated/torch.jit.trace.html) api that takes the model as a Python function or torch.nn.Module, and an example input  that will be passed to the function while tracing.

In [7]:
train_paths = os.path.join(OUTPUT_DIR, f"{1}/train.parquet")
dataset = Dataset(train_paths)

In [8]:
sparse_max = {'age_days-list': 20,
 'weekday_sin-list': 20,
 'item_id-list': 20,
 'category-list': 20}

from transformers4rec.torch.utils.data_utils import MerlinDataLoader

def generate_dataloader(schema, dataset, batch_size=128, seq_length=20):
    loader = MerlinDataLoader.from_schema(
            schema,
            dataset,
            batch_size=batch_size,
            max_sequence_length=seq_length,
            shuffle=False,
            sparse_as_dense=True,
            sparse_max=sparse_max
        )
    return loader

Create a dict of tensors to feed it as example inputs in the `torch.jit.trace()`.

In [9]:
loader = generate_dataloader(schema, dataset)
train_dict = next(iter(loader))

Let's check out the `item_id-list` column in the `train_dict` dictionary.

In [10]:
train_dict[0]['item_id-list']

tensor([[  7,  11,  73,  ...,   0,   0,   0],
        [ 42,  22,  30,  ...,   0,   0,   0],
        [  7,  60,   2,  ...,   0,   0,   0],
        ...,
        [ 18,  37,  18,  ...,   0,   0,   0],
        [ 12,  19,  33,  ...,   0,   0,   0],
        [ 11,  16, 102,  ...,   0,   0,   0]], device='cuda:0')

In [11]:
traced_model = torch.jit.trace(model, train_dict[0], strict=True)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Generate model input and output schemas to feed in the `PredictPyTorch` operator below.

In [12]:
input_schema = model.input_schema
output_schema = model.output_schema

In [13]:
for col_name, col_schema in input_schema.column_schemas.items():
    input_schema[col_name] = input_schema[col_name].with_shape((None, sparse_max[col_name]))

In [14]:
input_schema

,name,tags,dtype,is_list,is_ragged,properties.int_domain.min,properties.int_domain.max,properties.value_count.min,properties.value_count.max
0,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,False,0,0,20,20
1,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,False,0,0,20,20
2,item_id-list,"(Tags.ITEM_ID, Tags.ID, Tags.LIST, Tags.CATEGO...","DType(name='int64', element_type=<ElementType....",True,False,0,495,20,20
3,category-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,False,0,178,20,20


Let's create a folder that we can store the exported models and the config files.

In [15]:
import shutil
ens_model_path = os.environ.get("ens_model_path", f"{INPUT_DATA_DIR}/models")
# Make sure we have a clean stats space for Dask
if os.path.isdir(ens_model_path):
    shutil.rmtree(ens_model_path)
os.mkdir(ens_model_path)

We use `PredictPyTorch` operator that takes a pytorch model and packages it correctly for tritonserver to run on the PyTorch backend.

In [16]:
torch_op = input_schema.column_names >> PredictPyTorch(
    traced_model, input_schema, output_schema
)

The last step is to create the ensemble artifacts that Triton Inference Server can consume. To make these artifacts, we import the Ensemble class. The class is responsible for interpreting the graph and exporting the correct files for the server.

When we create an `Ensemble` object we supply the graph and a schema representing the starting input of the graph. The inputs to the ensemble graph are the inputs to the first operator of out graph. After we created the Ensemble we export the graph, supplying an export path for the `ensemble.export` function. This returns an ensemble config which represents the entire inference pipeline and a list of node-specific configs.

In [17]:
ensemble = Ensemble(torch_op, input_schema)
ens_config, node_configs = ensemble.export(ens_model_path)

In [18]:
ensemble.input_schema

,name,tags,dtype,is_list,is_ragged,properties.int_domain.min,properties.int_domain.max,properties.triton_scalar_shape,properties.value_count.min,properties.value_count.max
0,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,False,0,0,[],20,20
1,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,False,0,0,[],20,20
2,item_id-list,"(Tags.ITEM_ID, Tags.ID, Tags.LIST, Tags.CATEGO...","DType(name='int64', element_type=<ElementType....",True,False,0,495,[],20,20
3,category-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,False,0,178,[],20,20


## Starting Triton Server

It is time to deploy all the models as an ensemble model to Triton Inference Serve TIS. After we export the ensemble, we are ready to start the TIS. You can start triton server by using the following command on your terminal:

`tritonserver --model-repository=<ensemble_export_path>`

For the `--model-repository` argument, specify the same path as the export_path that you specified previously in the `ensemble.export` method. This command will launch the server and load all the models to the server. Once all the models are loaded successfully, you should see READY status printed out in the terminal for each loaded model.

In [19]:
import tritonclient.http as client

# Create a triton client
try:
    triton_client = client.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


After we create the client and verified it is connected to the server instance, we can communicate with the server and ensure all the models are loaded correctly.

In [20]:
# ensure triton is in a good state
triton_client.is_server_live()
triton_client.get_model_repository_index()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>
POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '121'}>
bytearray(b'[{"name":"0_predictpytorchtriton","version":"1","state":"READY"},{"name":"executor_model","version":"1","state":"READY"}]')


[{'name': '0_predictpytorchtriton', 'version': '1', 'state': 'READY'},
 {'name': 'executor_model', 'version': '1', 'state': 'READY'}]

### Send request to Triton and get the response

The last step of a machine learning (ML)/deep learning (DL) pipeline is to deploy the model to production, and get responses for a given query or a set of queries.
In this section, we generate a dataframe that we can serve as a request to TIS. Note that this is a transformed dataframe. We also need out dataset list columns to be padded to the max sequence length that was set in the ETL pipeline.

We do not serve the raw dataframe because in the production setting, we want to transform the input data as done during training (ETL). We need to apply the same mean/std for continuous features and use the same categorical mapping to convert the categories to continuous integer before we use the deployed DL model for a prediction. Therefore, we use a transformed dataset that is processed similarly as train set. 

In [21]:
eval_batch_size = 32
eval_paths = os.path.join(OUTPUT_DIR, f"{1}/valid.parquet")
eval_dataset = Dataset(eval_paths, shuffle=False)
eval_loader = generate_dataloader(schema, eval_dataset, batch_size=eval_batch_size)
test_dict = next(iter(eval_loader))

In [22]:
test_dict[0]['item_id-list'][0]

tensor([  8,   8,  10,  89, 130,  71,  16,  10,   9,   8,   8,  40,   0,   0,
          0,   0,   0,   0,   0,   0], device='cuda:0')

In [23]:
df_cols = {}
for name, tensor in test_dict[0].items():
    if name in input_schema.column_names:
        df_cols[name] = tensor.cpu().numpy()
        if len(tensor.shape) > 1:
            df_cols[name] = list(df_cols[name])
            
df = make_df(df_cols)
print(df.shape)
df.head()

(32, 4)


,age_days-list,weekday_sin-list,item_id-list,category-list
0,"[0.9501408, 0.49153143, 0.43261203, 0.60583574...","[0.50088716, 0.8589678, 0.25768423, 0.6750298,...","[8, 8, 10, 89, 130, 71, 16, 10, 9, 8, 8, 40, 0...","[4, 4, 5, 27, 38, 19, 3, 5, 2, 4, 4, 10, 0, 0,..."
1,"[0.8527362, 0.12124015, 0.882591, 0.45782763, ...","[0.47754368, 0.9635016, 0.2031468, 0.41193682,...","[5, 67, 25, 1, 30, 15, 10, 67, 38, 13, 19, 0, ...","[1, 17, 8, 1, 9, 3, 5, 17, 10, 5, 4, 0, 0, 0, ..."
2,"[0.40404844, 0.59667224, 0.011392503, 0.209335...","[0.9317083, 0.071744286, 0.958608, 0.24852555,...","[107, 13, 28, 18, 12, 91, 12, 2, 98, 1, 0, 0, ...","[33, 5, 7, 6, 4, 23, 4, 1, 26, 1, 0, 0, 0, 0, ..."
3,"[0.10275215, 0.5568824, 0.5089987, 0.14826113,...","[0.6575956, 0.71825516, 0.5113613, 0.3175862, ...","[8, 27, 10, 42, 6, 14, 12, 164, 77, 32, 0, 0, ...","[4, 7, 5, 12, 2, 3, 4, 51, 21, 4, 0, 0, 0, 0, ..."
4,"[0.87814647, 0.77221537, 0.20481698, 0.5081556...","[0.7751088, 0.00625885, 0.51996744, 0.73278934...","[12, 27, 2, 23, 5, 13, 26, 4, 19, 52, 0, 0, 0,...","[4, 7, 1, 8, 1, 5, 7, 2, 4, 13, 0, 0, 0, 0, 0,..."


Once our models are successfully loaded to the TIS, we can now easily send a request to TIS and get a response for our query with send_triton_request utility function.

In [24]:
from merlin.systems.triton.utils import send_triton_request
response = send_triton_request(input_schema, df[input_schema.column_names], output_schema.column_names)
print(response)

{'next-item': array([[ -9.942884 ,  -3.42258  ,  -3.4635031, ...,  -9.813356 ,
        -10.095871 ,  -9.584209 ],
       [ -9.943485 ,  -3.4231985,  -3.463529 , ...,  -9.813207 ,
        -10.095795 ,  -9.583893 ],
       [ -9.94396  ,  -3.4236832,  -3.463609 , ...,  -9.813052 ,
        -10.095815 ,  -9.583604 ],
       ...,
       [ -9.942841 ,  -3.4225972,  -3.4636211, ...,  -9.813358 ,
        -10.0957985,  -9.584264 ],
       [ -9.943155 ,  -3.4229283,  -3.4636493, ...,  -9.813265 ,
        -10.095781 ,  -9.584093 ],
       [ -9.943013 ,  -3.4227753,  -3.4636147, ...,  -9.813282 ,
        -10.09578  ,  -9.584151 ]], dtype=float32)}


In [25]:
response['next-item'].shape

(32, 496)

We return a response for each request in the df. Each row in the `response['next-item']` array corresponds to the logit values per item in the catalog, and one logit score corresponding to the null, OOV and padded items. The first score of each array in each row corresponds to the score for the padded item, OOV or null item. Note that we dont have OOV or null items in our syntheticall generated datasets.

This is the end of this suit of examples. You successfully performed feature engineering with NVTabular trained transformer architecture based session-based recommendation models with Transformers4Rec deployed a trained model to Triton Inference Server with Torch backend, sent request and got responses from the server. If you would like to learn how to serve a TF4Rec model with Python backend please visit this [example](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/end-to-end-session-based/02-End-to-end-session-based-with-Yoochoose-PyT.ipynb).